In [1]:
using Pkg; Pkg.activate("/home/dhairyagandhi96/temp/model-zoo/script/.."); Pkg.status();

using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

cd(@__DIR__)

isfile("input.txt") ||
  download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt",
           "input.txt")

text = collect(String(read("input.txt")))
alphabet = [unique(text)..., '_']
text = map(ch -> onehot(ch, alphabet), text)
stop = onehot('_', alphabet)

N = length(alphabet)
seqlen = 50
nbatch = 50

Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen))
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen))

m = Chain(
  LSTM(N, 128),
  LSTM(128, 128),
  Dense(128, N),
  softmax)

m = gpu(m)

function loss(xs, ys)
  l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
  Flux.truncate!(m)
  return l
end

opt = ADAM(0.01)
tx, ty = (Xs[5], Ys[5])
evalcb = () -> @show loss(tx, ty)

Flux.train!(loss, params(m), zip(Xs, Ys), opt,
            cb = throttle(evalcb, 30))
# Sampling
function sample(m, alphabet, len)
  m = cpu(m)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

sample(m, alphabet, 1000) |> println

    Status `~/temp/model-zoo/Project.toml`
  [1520ce14] AbstractTrees v0.2.1
  [fbb218c0] BSON v0.2.3
  [54eefc05] Cascadia v0.4.0
  [8f4d0f93] Conda v1.3.0
  [864edb3b] DataStructures v0.17.0
  [31c24e10] Distributions v0.21.3
  [587475ba] Flux v0.9.0
  [708ec375] Gumbo v0.5.1
  [b0807396] Gym v1.1.3
  [cd3eb016] HTTP v0.8.6
  [6218d12a] ImageMagick v0.7.5
  [916415d5] Images v0.18.0
  [e5e0dc1b] Juno v0.7.2
  [ca7b5df7] MFCC v0.3.1
  [91a5bcdd] Plots v0.26.3
  [2913bbd2] StatsBase v0.32.0
  [98b73d46] Trebuchet v0.1.0
  [8149f6b0] WAV v1.0.2
  [10745b16] Statistics 
  [4ec0a83e] Unicode 
loss(tx, ty) = 186.69904f0 (tracked)
loss(tx, ty) = 111.42951f0 (tracked)
loss(tx, ty) = 97.72145f0 (tracked)
loss(tx, ty) = 93.31382f0 (tracked)
loss(tx, ty) = 88.73668f0 (tracked)
loss(tx, ty) = 86.892075f0 (tracked)
loss(tx, ty) = 85.40693f0 (tracked)
loss(tx, ty) = 85.627556f0 (tracked)
loss(tx, ty) = 86.339386f0 (tracked)
loss(tx, ty) = 83.14823f0 (tracked)
loss(tx, ty) = 83.14904f0 (tracked)
&.